In [22]:
import pandas as pd

In [23]:
df = pd.read_csv("618.csv")
df

,content,time,name,QQ
0,@21-tongji-Frings @21-tongji-Frings 本科生在果壳里真是爹吧,2024-06-18 00:00:04,YhuoyuH,1398852396
1,感觉条件还是挺不错,2024-06-18 00:00:10,21-湖b-耐拒王蜀鼠,923557344
2,真是吧,2024-06-18 00:00:27,21-tongji-Frings,736455252
3,@YhuoyuH @YhuoyuH 真不是（,2024-06-18 00:00:29,21-ucas-tlx,1204096774
4,我们辅导员给我们还转了动物所的招生宣传,2024-06-18 00:00:37,21-uestc-安娜苏,2723980493
...,...,...,...,...
11410,哦哦哦，谢谢xd,2024-06-19 02:20:33,21-dhu-wrt,1186003117
11411,啊,2024-06-19 02:36:24,21-dlut-popo,2768849708
11412,沟槽的软工,2024-06-19 02:44:24,21-remake-4营0offer,1056813245
11413,破防了,2024-06-19 02:44:26,21-remake-4营0offer,1056813245


In [24]:
keywords = ['同济', 'tongji', 'tongj','黄渡理工','嘉定','tju','同济大学']  # 定义你关心的关键词

# 创建判断函数，判断一个字符串是否包含关键词中的任意一个
def contains_keywords(content):
    for word in keywords:
        if word in content:
            return True
    return False
contains_keywords("asd tongj")

True

In [27]:
mask = df['content'].str.contains('|'.join(keywords), case=False)
mask_filled = mask.fillna(False)
filtered_df = df[mask_filled]
filtered_df


,content,time,name,QQ
0,@21-tongji-Frings @21-tongji-Frings 本科生在果壳里真是爹吧,2024-06-18 00:00:04,YhuoyuH,1398852396
19,tongji上海无人中心那个报名表填意向导师是不是要提前联系下捏,2024-06-18 00:03:11,21-gpu-夏0营0ffer选手,3640519889
28,@20-uestc-pkurw-whiteflames @20-uestc-pkurw-wh...,2024-06-18 00:03:44,21-gpu-夏0营0ffer选手,3640519889
94,同济电院开奖了嘛,2024-06-18 00:10:12,21-su-禁止蕉绿,2067361394
147,同济入营,2024-06-18 00:13:22,19-nuist-tongji-ki,2944957728
...,...,...,...,...
10959,@19-nuist-tongji-ki 尤其是热门院校,2024-06-18 22:50:04,19-nuist-tongji-ki,2944957728
10967,@19-nuist-tongji-ki @19-nuist-tongji-ki 同济？,2024-06-18 22:50:30,21-西安职业学院-Ä,1657005350
11224,啊啊啊难受想拿fdu入营换tongji入营,2024-06-18 23:10:48,21 heu 粥,2910353465
11250,tju报名官网怎么这么糖,2024-06-18 23:12:43,22-gitu-浊水（六拒王）,1197962922


In [39]:
#内容拼接起来
content_txt = df['content'].str.cat(sep = '。')
content_txt = content_txt.replace('[图片]','').replace('[表情]','').replace('表情','')
#加载停用词
with open('cn_stopwords.txt',encoding = 'utf-8') as file:
    stopword = file.read()
import jieba
words = jieba.lcut(content_txt)
words = [word for word in words if (word not in stopword.split('\n'))&(len(word)>1)]
#大致看看是否已将无用信息去除完毕，若没有还需进行清洗
word_s = pd.Series(words).value_counts()
word_s


21        2764
同济         441
20         431
tongji     328
感觉         311
          ... 
还招           1
衣冠不整         1
不爱           1
说保           1
Fe           1
Length: 6705, dtype: int64

In [40]:
words = jieba.lcut(content_txt)

# 过滤停用词和长度小于等于1的词语
words = [word for word in words if (word not in stopword.split('\n')) and len(word) > 1]

# 定义一个函数用来判断是否全为数字
def contains_digits(word):
    return any(char.isdigit() for char in word)

# 过滤包含数字的词语
words = [word for word in words if not contains_digits(word)]

# 将列表转换为 Series 并统计词频
word_s = pd.Series(words).value_counts()

# 打印词频统计结果
word_s

同济        441
tongji    328
感觉        311
offer     304
本校        255
         ... 
八月份         1
笑鼠          1
能保          1
近战          1
Fe          1
Length: 6404, dtype: int64

In [41]:
#pycharts 里面提供的数据需要是 二维列表
data = word_s.reset_index().values.tolist()
#词云图 
import pyecharts.options as opts
from pyecharts.charts import WordCloud

(
    WordCloud()
    .add(series_name="QQ", data_pair=data, word_size_range=[20, 66])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="QQ", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    .render("tongji.html")
)



'e:\\大三下\\大数据与数据工程\\homework\\期末\\code\\tongji.html'